In [1]:
import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
sns.set() #make the plots look pretty
from bs4 import BeautifulSoup

In [2]:
f = io.open('data/list_candidates.xml', encoding='utf-8')
soup = BeautifulSoup(f, features='xml')
cand_list = soup.find_all('Candidat')
candidates = []

for c in cand_list:
    temp_dict = {}
    temp_dict['cand_id'] = c.find_all('NumPanneauCand')[0].text
    temp_dict['last_name'] = c.find_all('NomPsn')[0].text
    temp_dict['first_name'] = c.find_all('PrenomPsn')[0].text
    temp_dict['civilite'] = c.find_all('CivilitePsn')[0].text
    candidates.append(temp_dict)
f.close()
candidates = pd.DataFrame(candidates)

with io.open('data/results_pres_election.txt', encoding='iso-8859-1') as f:
    line = f.readline()
    
header = ['dpt_code','dpt_name','circo_code','circo_name','city_code','city_name','poll_station',
          'registered','abst','abst_perc','voting','voting_perc','white','white_perc_ins','white_perc_vot',
          'nullv', 'nullv_perc_ins', 'nullv_perc_vot','valid','valid_perc_ins','valid_perc_vot'
         ]

def pattern(i):
    tmp = ['cand_id','sex','last_name','first_name','votes','perc_ins','perc_exp']
    for j in range(len(tmp)):
        tmp[j] = str(i) + '_' + tmp[j]
    return tmp

for i in range(1,len(candidates)+1):
    header.extend(pattern(i))
    
results = pd.read_csv('data/results_pres_election.txt',
                      sep=';',
                      encoding='iso-8859-1',
                      header=0,
                      names = header,
                      dtype={'dpt_code': 'str',
                             'dpt_name': 'str',
                             'circo_code': 'int',
                             'circo_name': 'str',
                             'city_code':'str',
                             'city_name': 'str',
                             'poll_station': 'str',
                             'registered': 'int', 
                             'abst': 'int', 
                             'voting': 'int', 
                             'white': 'int',
                             'nullv': 'int', 
                             'valid': 'int', 
                             '1_votes': 'int', 
                             '2_votes': 'int', 
                             '3_votes': 'int', 
                             '4_votes': 'int', 
                             '5_votes': 'int',
                             '6_votes': 'int', 
                             '7_votes': 'int', 
                             '8_votes': 'int', 
                             '9_votes': 'int', 
                             '10_votes': 'int', 
                             '11_votes': 'int'
                            }
                     )

colums_to_keep = results.columns[[list(range(0,9)) + [10] + [12] + [15] + [18] + list(range(25,98,7))]]
results = results[colums_to_keep]

list_abroad_codes = sorted(list(set(results['dpt_code'].values)))[-7:]
results_abroad_idx = results['dpt_code'].isin(list_abroad_codes)
results_abroad = results[results_abroad_idx]
results_france = results[~results_abroad_idx]

results_france.loc[:,'dpt_code'] = results_france.loc[:,'dpt_code'].replace('Z.',u'97', regex=True, inplace=False)
results_france['insee_code'] = results_france['dpt_code'] + results_france['city_code']

/Users/aurelien/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/aurelien/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
income = pd.read_excel('data/income_by_city.xls', skiprows=5)[['CODGEO', 'MED13']]

In [4]:
insee_to_zip = pd.read_csv('data/insee_to_zipcode.csv', sep=';', dtype='str')

In [212]:
df = pd.merge(results_france, insee_to_zip, how='left', left_on='insee_code', right_on='insee_com')

In [220]:
len(df)

68308

### Clustering

In [215]:
from sklearn import preprocessing, pipeline, cluster

In [250]:
stds = preprocessing.StandardScaler(with_mean=True, with_std=True)
kms = cluster.KMeans(n_clusters=5, n_jobs=-2, n_init=10)

pipe = pipeline.Pipeline([
    ('kms', kms)
])

In [225]:
a = df.iloc[:,13:24]
b = df.iloc[:,12]

In [243]:
X = a.div(b, axis=0)

In [245]:
X[X['1_votes'].isnull()] = 0

In [248]:
X['abst'] = df.iloc[:,8]/df.iloc[:,7]

In [254]:
pipe.fit(X)
clusters = pipe.predict(X)

In [252]:
df_category = pd.concat([df['insee_code'], pd.Series(clusters)], axis=1)
df_category.columns = ['insee_code', 'group']
#df_category.to_csv('cities_grouped_by_code.csv')

In [253]:
df_category = pd.concat([df['city_name'], pd.Series(clusters)], axis=1)
df_category.columns = ['city_name', 'group']
#df_category.to_csv('cities_grouped_by_name.csv')

### Groupby on the groups

In [255]:
Y = pd.concat([df.iloc[:,[12,13,14,15,16,17,18,19,20,21,22,23]], pd.Series(clusters)], axis=1)

In [256]:
Y.columns = ['valid',  '1_votes',  '2_votes',  '3_votes',  '4_votes',  '5_votes',
        '6_votes',  '7_votes',  '8_votes',  '9_votes', '10_votes', '11_votes', 'group']

In [257]:
grouped = Y.groupby('group').sum()

In [258]:
grouped = grouped.reset_index()

In [259]:
a = grouped.iloc[:,-11:]
b = grouped.iloc[:,1]

In [260]:
result = a.div(b, axis=0)

In [261]:
results_by_group = result.iloc[:,[1, 2, 8, 10]]
results_by_group.columns = [candidates.last_name.iloc[[1, 2, 8, 10]]]

In [262]:
results_by_group

last_name,LE PEN,MACRON,MÉLENCHON,FILLON
0,0.349658,0.174402,0.188650,0.145365
1,0.258498,0.208979,0.149828,0.235421
2,0.191747,0.212734,0.325533,0.111643
3,0.160347,0.274588,0.225925,0.169296
4,0.106649,0.296759,0.131157,0.341969


### Trips data

In [263]:
trips = pd.read_csv('../Documents/Airbnb/Data/FR origin sample 2016 trips data - Aurelien Bocquet - d.27072017.csv')

In [264]:
trips_abroad = trips[trips['listing_country'] != 'FR']

In [267]:
groups_by_postal = pd.concat([df['postal_code'], pd.Series(clusters)], axis=1)
groups_by_postal.columns = ['user_postal_code', 'group']

In [274]:
#test = pd.merge(trips_abroad, groups_by_postal, how='left', left_on='user_postal_code', right_on='user_postal_code')

In [275]:
#print(len(test), len(trips_abroad))
#7638442 199872

In [276]:
test = trips_abroad.join(groups_by_postal, on='user_postal_code', lsuffix='_left', rsuffix='_right')

In [279]:
groups_by_postal.dtypes

user_postal_code    object
group                int32
dtype: object

In [280]:
trips_abroad.dtypes

Unnamed: 0                                 int64
id_guest_anon                             object
id_listing_anon                           object
listing_region                            object
listing_country                           object
m_bookings                                 int64
avg_daily_rate_in_usd                    float64
n_guests_on_booking                        int64
overall_rating_given_to_guest_by_host    float64
overall_rating_given_to_host_by_guest    float64
user_postal_code                          object
user_commune_name                         object
user_departement_name                     object
user_region_name                          object
listing_postal_code                       object
listing_commune_name                      object
listing_departement_name                  object
listing_region_name                       object
dtype: object

In [272]:
trips_abroad[trips_abroad['id_guest_anon'] == '738d3770-222f-4215-bcbe-8aca8c94f00d']

,Unnamed: 0,id_guest_anon,id_listing_anon,listing_region,listing_country,m_bookings,avg_daily_rate_in_usd,n_guests_on_booking,overall_rating_given_to_guest_by_host,overall_rating_given_to_host_by_guest,user_postal_code,user_commune_name,user_departement_name,user_region_name,listing_postal_code,listing_commune_name,listing_departement_name,listing_region_name
7,8,738d3770-222f-4215-bcbe-8aca8c94f00d,12685593-76cd-4412-9bf1-f10769b7a1e6,Europe,IE,1,74.0,2,NaN,NaN,92400,COURBEVOIE,HAUTS-DE-SEINE,ILE-DE-FRANCE,NaN,NaN,NaN,NaN
